In [105]:
import numpy as np
import pandas as pd

In [106]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
from sklearn.feature_selection import SelectKBest, chi2

In [107]:
df = sns.load_dataset('titanic')

In [108]:
df

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [109]:
df.drop(columns=['deck','class', 'who','adult_male','embark_town','alive','alone'], inplace=True)

In [110]:
cols = list(df.columns)
cols.remove('survived')
cols.append('survived')
df = df[cols]

In [111]:
df

,pclass,sex,age,sibsp,parch,fare,embarked,survived
0,3,male,22.0,1,0,7.2500,S,0
1,1,female,38.0,1,0,71.2833,C,1
2,3,female,26.0,0,0,7.9250,S,1
3,1,female,35.0,1,0,53.1000,S,1
4,3,male,35.0,0,0,8.0500,S,0
...,...,...,...,...,...,...,...,...
886,2,male,27.0,0,0,13.0000,S,0
887,1,female,19.0,0,0,30.0000,S,1
888,3,female,NaN,1,2,23.4500,S,0
889,1,male,26.0,0,0,30.0000,C,1


In [112]:
df.isnull().sum()

,0
pclass,0
sex,0
age,177
sibsp,0
parch,0
fare,0
embarked,2
survived,0


In [113]:
X_train, X_test, Y_train, Y_test = train_test_split(df.drop(columns=['survived']), df['survived'], test_size=0.2, random_state=42)

In [149]:
X_train.head(6)

,pclass,sex,age,sibsp,parch,fare,embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S
118,1,male,24.0,0,1,247.5208,C


In [115]:
Y_train.head()

,survived
331,0
733,0
382,0
704,0
813,0


In [150]:
trf1 = ColumnTransformer([
    ('impute_age', SimpleImputer(), [2]),
    ('impute_embarked', SimpleImputer(strategy='most_frequent'), [6])
], remainder='passthrough')

trf2 = ColumnTransformer([
    ('one_sex_embarked', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), [1,6]) # Targeting imputed_embarked (index 1) and sex (index 3) after trf1
], remainder='passthrough')

trf3 = ColumnTransformer([
    ('scale', MinMaxScaler(), slice(0,10))
])

In [135]:
trf4 = SelectKBest(score_func=chi2, k=8)

In [136]:
pipe = Pipeline([
    ('trf1', trf1),
    ('trf2', trf2),
    ('trf3', trf3),
    ('trf4', trf4),
])

In [137]:
from sklearn import set_config
set_config(display='diagram')

In [138]:
pipe.fit_transform(X_train, Y_train)

array([[0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.0556283 ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.02537431],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.01546857],
       ...,
       [0.        , 1.        , 0.        , ..., 0.25      , 0.        ,
        0.02753757],
       [0.        , 1.        , 1.        , ..., 0.125     , 0.33333333,
        0.2342244 ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.16666667,
        0.15085515]])

In [139]:
import pickle
pickle.dump(pipe, open('pipe.pkl', 'wb'))

In [140]:
import pickle

loaded_pipe = pickle.load(open('pipe.pkl', 'rb'))

In [151]:
test_input= np.array([3, 'male', 31, 0, 0, 15, 'S'], dtype=object).reshape(1,7)

In [152]:
loaded_pipe.transform(test_input)

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


array([[0.        , 1.        , 0.        , 1.        , 1.        ,
        0.        , 0.        , 0.02927805]])